In [29]:
# Jun 3rd 2022
# Author: Jose Bastias
# Description: Python code to add tide strain to the Mary Wildman dataset. 
# Actions: 
#          1.- Read the original the interpolated data set as pickle
#          2.- Compute volumetric strain
#          3.- Save

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from hampel import hampel
import pickle
import pygtide

plt.style.use('ggplot')
myColor  = ("#E3072A","#0A8974","#5FAECC","#E49C23","#E27520") # Darjeeling Color map

We load the data file that we generated with "02_data_interpolation.ipynb"

In [2]:
# Load results

path = 'Results/pkl_files/'

with open(path + 'interpolated_data.pkl', 'rb') as f:
    data = pickle.load(f)
data.keys()

dict_keys(['B1', 'B2'])

In [3]:
# Load well coordinates

path_resum = 'Baro_data/'

resum = pd.read_excel(path_resum + 'mary_wildman_metadata_summary2.xlsx')

keys = data.keys()
lati = np.zeros(len(keys))
long = np.zeros(len(keys))
height =np.zeros(len(keys))

for i, key in enumerate(keys):
    lati[i] = resum['Latitude'][i]
    long[i] = resum['Longitude'][i]
    height[i] = resum['Elevation_TOC [mAHD]'][i]

From here two options; use pygtide (Python wrapper of ETERNA) or the online tool (https://groundwater.app/app.php?app=pygtide)

In [ ]:
#%% PyGTide: Calculate volumetric strain variations
# To use Pygtide uncomment this section.

splrate = 3600

for i, key in enumerate(keys):
    #%% create a PyGTide object
    pt = pygtide.pygtide()
    
    # Now we need to deal with the offset of +9.5 hours
    data[key].index = data[key].index - np.timedelta64(int(3600 * 9.5),'s')
    if key == 'RN039769':
        data[key].index = data[key].index - np.timedelta64(int(3600 * 10.5),'s')
            
    # Now we come back to 3600s spacing
    # Resampling
    data[key] = data[key].resample('60T').mean()
    # Interpolation
    data[key] = data[key].interpolate(method='polynomial', order=3)
    
    duration = len(data[key]) + 24 # Add a day to fit the data
    start = data[key].index[0]
    
    pt.predict(lati[i], long[i], height[i], start, duration, splrate, tidalcompo=6)
    et_grav = pt.results()
    
    # Now we find the first va<lue of the fild data in the computed strain
    inx_init = np.where(et_grav['UTC'].values == data[key].index[0])[0]
    inx_end = np.where(et_grav['UTC'].values == data[key].index[-1])[0]
    
    # We add the strain to the data set
    data[key]['areal_strain'] = et_grav[inx_init[0]:inx_end[0]+1]['Signal [nstr]'].values

In [61]:
# Save data

path = 'Results/pkl_files/'

with open(path + 'areal_strain_data.pkl', 'wb') as f:
    pickle.dump(dic, f)